# How to use EBI Metagenomics API

The EMG REST API https://www.ebi.ac.uk/metagenomics/api/latest/ provides an easy-to-use set of top level resources, such as studies, samples, runs, experiment-types, biomes and annotations, that let user access metagenomics data in simple JSON format (JSON object formatted data structure is a syntax for storing and exchanging data). Retrieving the data is as simple as sending a HTTP request. Response return JSON object formatted data structure that contains the resource type, associated object identifier (id) with attributes. Where appropriate, relationships and links are provided to other resources.

We have utilised an interactive documentation framework (Swagger UI) to visualise and simplify interaction with the API’s resources via an HTML interface. Detailed explanations of the purpose of all resources, along with many examples, are provided to guide end-users. Documentation on how to use the endpoints is available at https://www.ebi.ac.uk/metagenomics/api/docs/.

# Browse API

### Task 1

Find marine studies

Answer:
1. https://www.ebi.ac.uk/metagenomics/api/latest/studies?lineage=root%3AEnvironmental%3AAquatic%3AMarine
2. https://www.ebi.ac.uk/metagenomics/api/latest/biomes/root:Environmental:Aquatic:Marine/studies

### Task 2

Find oceanic metagenomic samples taken from latitude >= 70° (N)

Answer:
1. https://www.ebi.ac.uk/metagenomics/api/latest/samples?experiment_type=metagenomic&lineage=root%3AEnvironmental%3AAquatic%3AMarine%3AOceanic&latitude_gte=70

2. https://www.ebi.ac.uk/metagenomics/api/latest/experiment-types/metagenomic/samples?experiment_type=&biome_name=&lineage=root%3AEnvironmental%3AAquatic%3AMarine%3AOceanic&geo_loc_name=&latitude_gte=70

# Write scripts

### Import Python modules

In [1]:
from pandas import DataFrame

try:
    from urllib import urlencode
except ImportError:
    from urllib.parse import urlencode

In [2]:
from jsonapi_client import Session, Filter

API_BASE = 'https://www.ebi.ac.uk/metagenomics/api/latest/'

### Get study

Get study: https://www.ebi.ac.uk/metagenomics/api/latest/studies/ERP009004

In [3]:
with Session(API_BASE) as s:
    study = s.get('studies', 'ERP009004').resource
    print('Study name:', study.study_name)
    print('Study abstract:', study.study_abstract)
    for biome in study.biomes:
        print('Biome:', biome.biome_name, biome.lineage)

Study name: Hydrocarbon Metagenomics Project
Study abstract: Metagenomics for Greener Production and Extraction of Hydrocarbon Energy:
Creating Opportunities for Enhanced Recovery with Reduced Environmental Impact
Biome: Freshwater root:Environmental:Aquatic:Freshwater
Biome: Marine root:Environmental:Aquatic:Marine
Biome: Soil root:Environmental:Terrestrial:Soil


### List samples with biomes for the given study

Get study: https://www.ebi.ac.uk/metagenomics/api/latest/studies/ERP001736

List samples: https://www.ebi.ac.uk/metagenomics/api/latest/studies/ERP001736/samples


Fetch samples for the given study accession: https://www.ebi.ac.uk/metagenomics/api/latest/samples?study_accession=ERP001736


In [4]:
df = DataFrame(columns=('sample name', 'lineage', 'biome', 'feature', 'material'))
df.index.name = 'accession'

with Session(API_BASE) as s:
    params = {
        'study_accession': 'ERP001736',
        'page_size': 100,
    }
    f = Filter(urlencode(params))
    for sample in s.iterate('samples', f):
        df.loc[sample.accession] = [
            sample.sample_name,
            sample.biome.id,
            sample.environment_biome,
            sample.environment_feature,
            sample.environment_material
        ]
df

,sample name,lineage,biome,feature,material
accession,,,,,
ERS488919,TARA_20100318T1133Z_039_EVENT_PUMP_P_D_(25 m)_...,root:Environmental:Aquatic:Marine:Oceanic,marine biome (ENVO:00000447),deep chlorophyll maximum layer (ENVO:xxxxxxxx),"saline water (ENVO:00002010), including plankt..."
ERS478017,TARA_20091215T1041Z_030_EVENT_PUMP_P_S_(5-7m)_...,root:Environmental:Aquatic:Marine:Oceanic,marine biome (ENVO:00000447),surface water (ENVO:00002042) layer,"saline water (ENVO:00002010), including plankt..."
ERS491463,TARA_20110312T1937Z_093_Combined-EVENTS_CAST_M...,root:Environmental:Aquatic:Marine:Oceanic,marine biome (ENVO:00000447),deep chlorophyll maximum layer (ENVO:xxxxxxxx),"""saline water (ENVO:00002010), including plank..."
ERS490542,TARA_20101016T0955Z_076_EVENT_PUMP_P_S_(5 m)_B...,root:Environmental:Aquatic:Marine:Oceanic,marine biome (ENVO:00000447),surface water layer (ENVO:00002042),"""saline water (ENVO:00002010), including plank..."
ERS490691,TARA_20101104T1816Z_078_Combined-EVENTS_CAST_M...,root:Environmental:Aquatic:Marine:Oceanic,marine biome (ENVO:00000447),deep chlorophyll maximum layer (ENVO:xxxxxxxx),"""saline water (ENVO:00002010), including plank..."
ERS490597,TARA_20101016T1700Z_076_Combined-EVENTS_CAST_M...,root:Environmental:Aquatic:Marine:Oceanic,marine biome (ENVO:00000447),deep chlorophyll maximum layer (ENVO:xxxxxxxx),"""saline water (ENVO:00002010), including plank..."
ERS489315,TARA_20100419T0756Z_048_EVENT_PUMP_P_S_(5 m)_B...,root:Environmental:Aquatic:Marine:Oceanic,marine biome (ENVO:00000447),surface water layer (ENVO:00002042),"""saline water (ENVO:00002010), including plank..."
ERS491095,TARA_20110106T1936Z_085_Combined-EVENTS_CAST_M...,root:Environmental:Aquatic:Marine:Oceanic,marine biome (ENVO:00000447),deep chlorophyll maximum layer (ENVO:xxxxxxxx),"""saline water (ENVO:00002010), including plank..."
ERS492778,TARA_20110801T1755Z_123_Combined-EVENTS_CAST_M...,root:Environmental:Aquatic:Marine:Oceanic,marine biome (ENVO:00000447),marine epipelagic mixed layer (ENVO:xxxxxxxxx),"""saline water (ENVO:00002010), including plank..."


### List samples with biomes and metadata for the given study

Samples for the given study accession: https://www.ebi.ac.uk/metagenomics/api/latest/samples?study_accession=ERP001736


In [5]:
def get_metadata(metadata, key):
    import html
    for m in metadata:
        if m['key'].lower() == key.lower():
            value = m['value']
            unit = html.unescape(m['unit']) if m['unit'] else ""
            return "{value} {unit}".format(value=value, unit=unit)
    return None

depth_label = 'geographic location (depth)'
temp_label = 'temperature'
df = DataFrame(columns=('sample name', 'biome', 'temperature', 'depth', 'longitude', 'latitude'))
df.index.name = 'accession'

with Session(API_BASE) as s:
    params = {
        'study_accession': 'ERP001736',
        'include': 'biome',
        'page_size': 100,
    }
    f = Filter(urlencode(params))
    for sample in s.iterate('samples', f):
        df.loc[sample.accession] = [
            sample.sample_name, sample.biome.id,
            get_metadata(sample.sample_metadata, temp_label),
            get_metadata(sample.sample_metadata, depth_label),
            sample.longitude, sample.latitude
        ]
df

,sample name,biome,temperature,depth,longitude,latitude
accession,,,,,,
ERS488919,TARA_20100318T1133Z_039_EVENT_PUMP_P_D_(25 m)_...,root:Environmental:Aquatic:Marine:Oceanic,26.812225 °C,25 m,66.4727,18.5839
ERS478017,TARA_20091215T1041Z_030_EVENT_PUMP_P_S_(5-7m)_...,root:Environmental:Aquatic:Marine:Oceanic,20.460612 °C,5 m,32.8980,33.9179
ERS491463,TARA_20110312T1937Z_093_Combined-EVENTS_CAST_M...,root:Environmental:Aquatic:Marine:Oceanic,16.40115 °C,35 m,-73.0537,-33.9116
ERS490542,TARA_20101016T0955Z_076_EVENT_PUMP_P_S_(5 m)_B...,root:Environmental:Aquatic:Marine:Oceanic,23.349542 °C,5 m,-35.1803,-20.9354
ERS490691,TARA_20101104T1816Z_078_Combined-EVENTS_CAST_M...,root:Environmental:Aquatic:Marine:Oceanic,19.30925 °C,120 m,-43.2705,-30.1484
ERS490597,TARA_20101016T1700Z_076_Combined-EVENTS_CAST_M...,root:Environmental:Aquatic:Marine:Oceanic,21.643283 °C,150 m,-35.3498,-21.0292
ERS489315,TARA_20100419T0756Z_048_EVENT_PUMP_P_S_(5 m)_B...,root:Environmental:Aquatic:Marine:Oceanic,29.818233 °C,5 m,66.4228,-9.3921
ERS491095,TARA_20110106T1936Z_085_Combined-EVENTS_CAST_M...,root:Environmental:Aquatic:Marine:Oceanic,-0.78154 °C,90 m,-49.2139,-62.2231
ERS492778,TARA_20110801T1755Z_123_Combined-EVENTS_CAST_M...,root:Environmental:Aquatic:Marine:Oceanic,22.1151 °C,150 m,-140.2845,-8.9109


### List runs

Get sample: https://www.ebi.ac.uk/metagenomics/api/latest/samples/ERS667565

List runs: https://www.ebi.ac.uk/metagenomics/api/latest/samples/ERS667565/runs

In [6]:
df = DataFrame(columns=('instrument platform', 'instrument model', 'analysis pipeline'))
df.index.name = 'accession'

with Session(API_BASE) as s:
    sample = s.get('samples', 'ERS667565').resource
    for run in sample.runs:
        df.loc[run.accession] = [
            run.instrument_platform, run.instrument_model,
            ", ".join([p.release_version for p in run.pipelines])
        ]

df

,instrument platform,instrument model,analysis pipeline
accession,,,
ERR867951,ILLUMINA,Illumina MiSeq,4.0
ERR867950,ILLUMINA,Illumina MiSeq,4.0
ERR771104,None,None,"2.0, 4.0"


### List sample metadata

Get sample with metadata: https://www.ebi.ac.uk/metagenomics/api/latest/samples/ERS488919

In [7]:
def format_unit(unit):
    import html
    return html.unescape(unit) if unit else ""

df = DataFrame(columns=('metadata key', 'value', 'unit'))

with Session(API_BASE) as s:
    sample = s.get('samples', 'ERS488919').resource
    print(sample.sample_name, sample.accession)
            
    for i, m in enumerate(sample.sample_metadata):
        df.loc[i] = [
            m['key'], m['value'],
            format_unit(m['unit'] or None)
        ]

df

TARA_20100318T1133Z_039_EVENT_PUMP_P_D_(25 m)_BACT_NUC-DNA(100L)_W1.6-20_TARA_B100000105 ERS488919


,metadata key,value,unit
0,temperature,26.812225,°C
1,project name,Tara Oceans expedition (2009-2013),
2,geographic location (depth),25,m
3,environmental package,water,
4,instrument model,Illumina HiSeq 2000,
5,ENA checklist,ENA TARA (ERC000030),
6,latitude end,18.5679,DD
7,longitude end,66.4581,DD
8,marine region,n/a,
9,protocol label,BACT_NUC-DNA(100L)_W1.6-20,


### List functional annotations

Gene Ontology (GO) terms derived from InterPro matches: https://www.ebi.ac.uk/metagenomics/api/latest/runs/ERR263024/pipelines/1.0/go-slim

In [8]:
df = DataFrame(columns=('category', 'description', 'annotation counts'))
df.index.name = 'GO term'

with Session(API_BASE) as s:
    f = Filter('page_size=100')
    for ann in s.iterate('runs/SRR1047054/pipelines/2.0/go-slim', f):
        df.loc[ann.accession] = [
            ann.lineage, ann.description, ann.count
        ]
df

,category,description,annotation counts
GO term,,,
GO:0000156,molecular_function,two-component response regulator activity,2.0
GO:0000160,biological_process,phosphorelay signal transduction system,50.0
GO:0000166,molecular_function,nucleotide binding,316.0
GO:0000746,biological_process,conjugation,3.0
GO:0000902,biological_process,cell morphogenesis,5.0
GO:0000988,molecular_function,protein binding transcription factor activity,17.0
GO:0001071,molecular_function,nucleic acid binding transcription factor acti...,47.0
GO:0002376,biological_process,immune system process,0.0
GO:0003676,molecular_function,nucleic acid binding,218.0


### List marine metagenomic samples collected in a temperature between 1°C and 5°C

List samples: https://www.ebi.ac.uk/metagenomics/api/latest/biomes/root:Environmental:Aquatic:Marine/samples?experiment_type=metagenomic&metadata_key=temperature&metadata_value_gte=1&metadata_value_lte=5

In [9]:
def get_metadata(metadata, key='temperature'):
    import html
    for m in metadata:
        if m['key'].lower() == key.lower():
            value = m['value']
            unit = html.unescape(m['unit']) if m['unit'] else ""
            return "{value} {unit}".format(value=value, unit=unit)
    return None

depth_label = 'geographic location (depth)'
temp_label = 'temperature'
df = DataFrame(columns=('sample name', 'biome', 'temperature', 'depth', 'location', 'latitude'))
df.index.name = 'accession'

with Session(API_BASE) as s:
    params = {
        'experiment_type': 'metagenomic',
        'metadata_key': 'temperature',
        'metadata_value_gte': 1,
        'metadata_value_lte': 5,
        'latitude_gte': 0,
        'include': 'biome',
    }
    f = Filter(urlencode(params))
    for sample in s.iterate('biomes/root:Environmental:Aquatic:Marine/samples', f):
        df.loc[sample.accession] = [
            sample.sample_name, sample.biome.id,
            get_metadata(sample.sample_metadata, temp_label),
            get_metadata(sample.sample_metadata, depth_label),
            sample.geo_loc_name, sample.latitude
        ]
df

,sample name,biome,temperature,depth,location,latitude
accession,,,,,,
SRS981327,BP_381106-1,root:Environmental:Aquatic:Marine,4.4889,None,USA: Gulf of Mexico,28.7051
SRS981330,BP_381105-1,root:Environmental:Aquatic:Marine,5.5365,None,USA: Gulf of Mexico,28.7051
SRS981311,BP_381111-2,root:Environmental:Aquatic:Marine,4.4199,None,USA: Gulf of Mexico,28.6632
SRS981308,BP_381112-2,root:Environmental:Aquatic:Marine,4.281,None,USA: Gulf of Mexico,28.6632
SRS981307,BP_381112-3,root:Environmental:Aquatic:Marine,4.281,None,USA: Gulf of Mexico,28.6632
SRS981326,BP_381106-2,root:Environmental:Aquatic:Marine,4.4889,None,USA: Gulf of Mexico,28.7051
SRS981325,BP_381106-3,root:Environmental:Aquatic:Marine,4.4889,None,USA: Gulf of Mexico,28.7051
SRS981309,BP_381112-1,root:Environmental:Aquatic:Marine,4.281,None,USA: Gulf of Mexico,28.6632
SRS981322,BP_381107-3,root:Environmental:Aquatic:Marine,4.2599,None,USA: Gulf of Mexico,28.7051


### Export to CSV

Get study: https://www.ebi.ac.uk/metagenomics/api/latest/studies/ERP005831

In [10]:
import csv

with open("output.csv", "w") as csvfile:
    with Session(API_BASE) as s:
        fieldnames = ['study', 'sample', 'biome', 'lineage', 'longitude', 'latitude']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        study = s.get('studies', 'ERP005831').resource
        for sample in study.samples:
            biome = sample.biome
            row = {
                'study': study.accession,
                'sample': sample.accession,
                'biome': biome.biome_name,
                'lineage': biome.lineage,
                'longitude': sample.longitude,
                'latitude': sample.latitude
            }
            writer.writerow(row)

df = DataFrame().from_csv('output.csv')
df

,sample,biome,lineage,longitude,latitude
study,,,,,
ERP005831,ERS456668,Sediment,root:Environmental:Aquatic:Freshwater:Lentic:S...,-1.56,52.38
ERP005831,ERS456669,Agricultural,root:Environmental:Terrestrial:Soil:Loam:Agric...,-1.61,52.19
